In [ ]:
from datetime import datetime, timedelta
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline
%config IPCompleter.greedy = True
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
import cbpro
import os
from pathlib import Path
import seaborn as sns
import json
import urllib
import requests

# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import tensorflow_probability as tfp

tf.distribute.OneDeviceStrategy(device="/gpu:0")
policy = tf.keras.mixed_precision.Policy("mixed_float16")
tf.keras.mixed_precision.experimental.set_policy(policy)

public_client = cbpro.PublicClient()


In [ ]:
class MinerMeta(type):
    # def __init__(self):
    #     result = getattr(self, "df", None)
    #     if result is None:
    #         self.df = self.compile_historic(read_csv=True)

    def compile_historic(self, num_days=100, write_csv=False, read_csv=False):
        file = Path.cwd() / f"{self.coin}_histdata.csv"
        if read_csv is True:
            df = pd.read_csv(file, index_col="time", infer_datetime_format=True)
            return df
        else:
            finish = datetime.now()
            start = finish - timedelta(num_days)
            delta = timedelta(hours=300)
            df = pd.DataFrame()

            while finish > start:
                historic = public_client.get_product_historic_rates(
                    f"{self.coin}-USD",
                    granularity=3600,
                    start=start,
                    end=start + delta,
                )
                start += delta
                df = df.append(historic, ignore_index=True, verify_integrity=True)
            df.columns = ["time", "low", "high", "open", "close", "volume"]

            # timestamp_s = date_time.map(pd.Timestamp.timestamp)
            # day = 24 * 60 * 60
            # year = (365.2425) * day
            # df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
            # df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
            # df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
            # df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))

            df["time"] = pd.to_datetime(df["time"], unit="s")
            df.set_index("time", inplace=True, verify_integrity=False)
            df.sort_index(ascending=False, inplace=True)

            if write_csv is True:
                df.to_csv(file, index=True)

            return df

    def get_day_stats(self):
        result = getattr(self, "day_stats", None)
        if result is None:
            ticker = public_client.get_product_24hr_stats(f"{self.coin}-USD")
            df = pd.DataFrame.from_dict(ticker, orient="index")
            self.day_stats = df
        return df

    def year_day_fft(self, col):
        df = self.compile_historic()
        today = datetime.today()
        ylim = int(df[col].max())
        plt.figure(figsize=(15, 10))

        fft = tf.signal.rfft(df[col])
        f_per_dataset = np.arange(0, len(fft))

        n_samples_h = 1
        hours_per_year = 24 * 365.2524
        hours_per_week = 24 * 7
        years_per_dataset = n_samples_h / (hours_per_year)
        hours_per_dataset = n_samples_h / (hours_per_week)

        f_per_year = f_per_dataset / years_per_dataset
        f_per_week = f_per_dataset / hours_per_dataset
        plt.step(f_per_week, np.abs(fft))
        plt.xscale("log")
        # plt.ylim(1000, ylim)
        # plt.xlim([0.1, max(plt.xlim())])
        plt.xticks([1, 7], labels=["1/Week", "1/day"])
        plt.xlabel("Frequency (log scale)")

        return plt.show()

    def ttsplit_norm(self, df, split_time=0.7, feature_plot=False):
        # train_df Test Split
        n = len(df)
        train_df = df[0 : int(n * 0.7)]
        val_df = df[int(n * 0.7) : int(n * 0.9)]
        test_df = df[int(n * 0.9) :]
        # Normalize the Data
        train_df_mean = train_df.mean()
        train_df_std = train_df.std()

        train_df = (train_df - train_df_mean) / train_df_std
        val_df = (val_df - train_df_mean) / train_df_std
        test_df = (test_df - train_df_mean) / train_df_std

        # Create Feature Plot if wanted
        if feature_plot is True:
            df_std = (df - train_df_mean) / train_df_std
            df_std = df_std.melt(var_name="Column", value_name="Normalized")
            plt.figure(figsize=(12, 6))
            ax = sns.violinplot(x="Column", y="Normalized", data=df_std)
            ax.set_xticklabels(df.keys(), rotation=90)
            ax.set_title("train_dfing Data Feature Dist with whole DF Mean")

            return train_df, val_df, test_df

        return train_df, val_df, test_df

    def __call__(self, *args, **kwargs):

        cls = type.__call__(self, *args)

        # setattr(cls, "compile_historic", self.compile_historic)
        # setattr(cls, "year_day_fft", self.year_day_fft)
        # setattr(cls, "ttsplit_norm", self.ttsplit_norm)
        # setattr(cls, "get_day_stats", self.get_day_stats)
        # setattr(cls, "day_stats", self.get_day_stats())

        # for key, value in historic.items():
        #     setattr(cls, "hist_" + key, value)
        # for key, value in ticker.items():
        #     setattr(cls, "tick_" + key, value)

        return cls


In [ ]:
class eth(metaclass=MinerMeta):
    coin = "eth"


df = eth.compile_historic(read_csv=True)
print(len(df))
df = df[:1000]
print(len(df))
df.head()


In [ ]:
api_key = r"ru1zaf0ssaa29394mb4ahp"
lc_file = Path.cwd() / "lc_data.csv"
coins = "ETH"


def lc_data(read_csv=False):
    file = Path.cwd() / "lunardata.csv"
    if read_csv is True:
        data = pd.read_csv(file, index_col="id")
        return data
    else:
        assets_url = (
            b"https://api.lunarcrush.com/v2?data=assets&key="
            + api_key
            + "&symbol="
            + coins
            + 
            + '&data_points=720'

        )
        assets = json.loads(urllib.request.urlopen(assets_url).read())
        # config = pd.DataFrame.from_dict(assets["config"], orient="index")
        # usage = pd.DataFrame.from_dict(assets["usage"], orient="index")
        data = pd.DataFrame.from_dict(assets["data"])
        return data


lc = lc_data()

lc.tail()

In [46]:
import requests

intervals = ["1d", "1w", "1m", "3m", "6m", "1y", "2y"]
now = datetime.now()
print(now)
delta = timedelta(100)
print(delta)
ut = datetime.timestamp((now - delta))
print(ut)

payload = {
    "key": api_key,
    "symbol": coins,
    "data_points": "720",
    "change": intervals,
    # "start": ut,
    # "end": datetime.timestamp(now),
}

r = requests.get("https://api.lunarcrush.com/v2?data=assets", params=payload)

print(r.url)
r.json()

2021-10-07 12:49:38.235071
100 days, 0:00:00
1624996178.235071
https://api.lunarcrush.com/v2?data=assets&key=ru1zaf0ssaa29394mb4ahp&symbol=ETH&data_points=720&change=1d&change=1w&change=1m&change=3m&change=6m&change=1y&change=2y


{'config': {'change': '2y',
  'data': 'assets',
  'data_points': '720',
  'symbol': "'ETH'"},
 'usage': {'day': 16, 'month': 16},
 'data': [{'id': 2,
   'name': 'Ethereum',
   'symbol': 'ETH',
   'price': 3606.13565458,
   'price_btc': 0.06660860438770307,
   'market_cap': 424924092312,
   'percent_change_24h': 0.46,
   'percent_change_7d': 20.16,
   'percent_change_30d': 4.64,
   'volume_24h': 31514809369.2,
   'max_supply': '113561765',
   'categories': None,
   'timeSeries': [{'asset_id': 2,
     'time': 1631041200,
     'open': 3461.71957034,
     'close': 3425.92377361,
     'high': 3485.52940215,
     'low': 3417.46815236,
     'volume': 1875988544.97,
     'market_cap': 403952243724,
     'url_shares': 1073,
     'unique_url_shares': 652,
     'reddit_posts': 55,
     'reddit_posts_score': 362,
     'reddit_comments': 218,
     'reddit_comments_score': 447,
     'tweets': 4192,
     'tweet_spam': 1832,
     'tweet_followers': 14936034,
     'tweet_quotes': 87,
     'tweet_retwee

In [ ]:
def plot_loss(history):
    plt.plot(history.history["loss"], label="loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.ylim([0, 10])
    plt.xlabel("Epoch")
    # plt.ylabel("Error [MPG]")
    plt.legend()
    plt.grid(True)


In [ ]:
cols = list(df.columns)

# sns.pairplot(df[cols], diag_kind="kde")

In [ ]:
def tt_split(df, train_percent=0.6, val_percent=0.2):
    m = len(df.index)
    train_end = int(train_percent * m)
    val_end = int(val_percent * m) + train_end
    train = df.iloc[:train_end]
    # val = df.iloc[train_end:val_end]
    # test = df.iloc[val_end:]
    test = df.iloc[train_end:]
    return train, test


train, test = tt_split(df)

label = train.pop("close")
test_label = test.pop("close")


def make_ds(x, y):
    ds = (
        tf.data.Dataset.from_tensor_slices((x.values, y.values))
        .batch(24)
        .prefetch(tf.data.AUTOTUNE)
    )
    return ds


ds = make_ds(train, label)
# val_ds = make_ds(val, val_label)
test_ds = make_ds(test, test_label)

for features_tensor, target_tensor in ds.take(1):
    print(f"features:{features_tensor} target:{target_tensor}")


In [ ]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)
normalizer.adapt(train.values)


In [ ]:
linear_model = tf.keras.Sequential([normalizer, tf.keras.layers.Dense(units=1)])
linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1), loss="mean_absolute_error"
)

linear_model.summary()


In [ ]:
%time
history = linear_model.fit(
    train.values, label, 
    epochs=100,
    validation_split = 0.2)

In [ ]:
hist = pd.DataFrame(history.history)
hist["epoch"] = history.epoch
hist.tail()


In [ ]:
plot_loss(history)


In [ ]:
model = tf.keras.Sequential(
    [
        normalizer,
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1), loss="mean_absolute_error"
)


model.summary()


In [ ]:
%%time
history = model.fit(train.values, label.values,epochs=100, validation_split=.2)

In [ ]:
hist = pd.DataFrame(history.history)
hist["epoch"] = history.epoch
hist.tail()


In [ ]:
plot_loss(history)


In [ ]:
yhat2 = model.predict(test[:1])
print(yhat2)